In [11]:
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
from typing import List, Optional

from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

import pandas as pd
from pydantic import BaseModel, Field, validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain


from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from utils import load_conversation

In [13]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
)

In [14]:
conversation = load_conversation('emails/sample_4.txt')
doc = Document(page_content=conversation)
split_docs = RecursiveCharacterTextSplitter().split_documents([doc])

In [30]:
with open(r"emails/sample_4.txt", encoding="utf-8") as f:
    conversation = f.read()

In [ ]:
from langchain.chat_models import ChatOpenAI

prompt = f"""

Your goal is to extract structured information from the email chain below that matches the form described below. 
When extracting information, please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

personal_info: Array<{{ // Contact information for a person
 first_name: string // The first name of the person
 last_name: string // The last name of the person
 job_title: string // The job title of the person
 company_name: string // The company name the person works for
 mobile_number: string // The mobile or direct number of the person
 desk_number: string // The desk or office number of the person
 business_website: string // The business website of the company that the person works for
 email: string // The email of the person
 address: string // The address of the company the person works for
}}>

Please output the extracted information in JSON format.
Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.

EMAIL CHAIN:
{conversation}

"""

llm = ChatOpenAI()

output = llm.predict(prompt,)

import pandas as pd
import json
pd.DataFrame(json.loads(output))

In [ ]:
json.loads(output)

In [40]:
from pydantic import BaseModel, Field
from typing import Optional

from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage

from typing import List
from pydantic import BaseModel

class ContactInfo(BaseModel):
    first_name: str
    last_name: str
    job_title: str
    company_name: str
    mobile_number: str
    desk_number: str
    business_website: str
    email: str
    address: str

class PersonalInfo(BaseModel):
    personal_info: List[ContactInfo]


# If we pass in a model explicitly, we need to make sure it supports the OpenAI function-calling API.
llm = ChatOpenAI(model="gpt-4", temperature=0)

prompt_msgs = [
    SystemMessage(
        content="You are a world class algorithm for extracting information in structured formats."
    ),
    HumanMessage(
        content="Use the given format to extract information from the following input:"
    ),
    HumanMessagePromptTemplate.from_template("{input}"),
    HumanMessage(content="Tips: Make sure to answer in the correct format"),
]
prompt = ChatPromptTemplate(messages=prompt_msgs)

chain = create_structured_output_chain(PersonalInfo, llm, prompt, verbose=True)
chain.run(conversation)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class algorithm for extracting information in structured formats.
Human: Use the given format to extract information from the following input:
Human: Sally is 13
Human: Tips: Make sure to answer in the correct format

> Finished chain.


Person(name='Sally', age=13, fav_food='Unknown')